In [2]:
from dynamics import *
import numpy as np
import pandas as pd

file_path   = 'bvolmatrix.xlsx'
sheet       = 'STRADDLE'
sheetf      = 'STRIKE'
df          = pd.read_excel(file_path, sheet_name=sheet,header=None)
dff         = pd.read_excel(file_path, sheet_name=sheetf,header=None)
straddlesobs       = pd.DataFrame(df[[x for x in np.arange(4,16)]][3:22])
strikes            = pd.DataFrame(dff[[x for x in np.arange(4,16)]][3:22])
straddlesobs.index     = df[1][3:22].values
strikes.index          = dff[1][3:22].values
straddlesobs.index.name='Expiry'
strikes.index.name     ='Expiry' 
straddlesobs.columns   = [int(x) for x in df.iloc[1][4:16].values]
strikes.columns        = [int(x) for x in dff.iloc[1][4:16].values]
# straddlesobs
sheeta      = 'ARTICLE'
dfff        = pd.read_excel(file_path, sheet_name=sheeta,header=None)
articleobs  = dfff[[x for x in np.arange(4,17)]][3:17]/10000
articleobs.index = dfff[1][3:17].values
articleobs.index.name = 'Expiry'
articleobs.columns = [int(x) for x in dfff.iloc[1][4:17].values]

straddlesobs


,12,24,36,48,60,84,120,144,180,240,300,360
Expiry,,,,,,,,,,,,
1,37.926044,44.260104,42.184836,40.222429,37.677289,34.970918,30.769709,30.311353,29.585816,28.443382,27.528974,26.653722
3,62.183133,69.535639,67.321510,63.607357,60.152517,56.394481,50.749447,50.123108,49.113784,47.557909,46.408955,45.355748
6,82.202422,87.517088,85.768100,82.969720,79.923096,75.449073,68.351568,67.448865,65.987614,63.979099,62.749165,61.270989
9,96.337589,100.580257,98.569481,95.701879,93.145223,88.944014,81.343143,80.341210,78.682838,76.098541,74.764934,72.864716
12,107.522923,110.706483,108.296871,105.520233,102.767531,98.714278,91.916301,90.703517,88.668911,84.966727,83.458725,81.735294
24,135.890703,137.030366,134.830027,132.956917,131.490024,128.556239,123.828330,121.639274,117.949474,113.627782,111.246902,109.768725
36,156.702326,157.213658,155.693484,153.219745,151.008583,148.161711,143.324793,140.657578,136.014137,130.900823,128.648201,126.119184
48,173.332442,174.098411,171.241984,168.928119,166.837662,163.247181,157.821566,154.406620,149.108667,143.268152,140.220233,136.805287
60,187.404397,187.315191,184.335704,181.213486,178.465935,174.451656,168.724618,164.746021,159.018982,152.542612,148.653221,144.870878


In [5]:
from dynamics import *
from Vasicek import *
def VasicekOptimizer(initial,mean,reversion,volatility, observedP, observedK):
    straddleVasi = observedP.copy()
    optVasi = Vasicek(initial=initial, mean=mean, reversion=reversion, volatility=volatility)
    # optVasi = Vasicek(initial=initial, mean=0.02, reversion=0.95, volatility=volatility)
    for expiry in observedP.index:
        # Expiry first
        # Getting the right expiry and converting to years
        
        for tenor in observedP.columns:
            # print(expiry,tenor)
            # Creating tenor legs
            T = np.arange(expiry/12, expiry/12+tenor/12+0.5, 0.5)
            S = np.arange(expiry/12, expiry/12+tenor/12+1)
            # Finding the ATM fixed rate
            K = observedK[tenor].loc[expiry]/100
            print(f'Expiry:{expiry}, Tenor:{tenor}, Strike:{K}, swap:{optVasi.swap(0,expiry/12,expiry/12+tenor/12, K)}')
            #Calculating the payer and receiver swpt price
            p = optVasi.swaptionSC(0,expiry/12, fixedSchedule=S, floatingSchedule=T, fixedRate=K, payer=True)
            r = optVasi.swaptionSC(0,expiry/12, fixedSchedule=S, floatingSchedule=T, fixedRate=K, payer=False)
            # print(f'Expiry:{expiry}, Tenor:{tenor}, Straddle:{p+r}')
            straddleVasi[tenor].loc[expiry] = (p+r)*10000
            # print(f'Expiry: {expiry}, Tenor: {tenor} fixedrate: {K}')
            # print(f'T={T}')
            # print(f'S={S}')
            # print(f'Price:{p+r}')
    error = ((straddleVasi-observedP)**2).sum().sum()
    # print(f'{error}', end='                                                         \r')
    return error,straddleVasi

In [6]:
#Trying to fit all prices (failing)
from scipy.optimize import minimize, basinhopping
EURIBORFIXMARCH22 = 0.03071
Nfeval = 0
def callback(xk):
    global Nfeval
    print(f'Eval: {Nfeval}, x={xk}, func: {VasicekOptimizer(initial=0.03071, mean=xk[0], reversion=xk[1], volatility=xk[2],observedP=straddlesobs, observedK=strikes)[0]}')
    Nfeval += 1
    return True
# callback([0.027832947, 0.11874695, 0.019677])

result = minimize(
    # fun = lambda x: VasicekOptimizer(EURIBORFIXMARCH22, x[0], x[1], x[2], observed=straddlesobs)[0],
    fun = lambda x: VasicekOptimizer(EURIBORFIXMARCH22, x[0], x[1], x[2], observedP=straddlesobs, observedK=strikes)[0],
    x0  = [0.06460112, 2.69262944, 0.08495089],
    bounds = [(0.0001,1),(0.0001,1),(0.0001,1)],
    method='nelder-mead',
    callback=callback
)

C:\Users\Shark\AppData\Local\Temp\ipykernel_24628\3391481147.py:12: OptimizeWarning: Initial guess is not within the specified bounds
  result = minimize(


TypeError: 'float' object is not subscriptable

In [11]:
result

 final_simplex: (array([[0.02380454, 1.        , 0.02850604],
       [0.02380456, 1.        , 0.02850629],
       [0.02380456, 1.        , 0.02850623],
       [0.02380454, 1.        , 0.02850576]]), array([28759630.6579874 , 28759630.65799707, 28759630.65800584,
       28759630.65804101]))
           fun: 28759630.657987405
       message: 'Optimization terminated successfully.'
          nfev: 166
           nit: 86
        status: 0
       success: True
             x: array([0.02380454, 1.        , 0.02850604])

In [53]:
res.x

array([4.36560647e-05])

In [71]:
from scipy.optimize import minimize, basinhopping
VasiImpVol      = straddlesobs.copy()
VasiImpVolPrice = straddlesobs.copy()
for expiry in straddlesobs.index:
        # Expiry first
        # Getting the right expiry and converting to years
        
        for tenor in straddlesobs.columns:
            
            res = minimize(
                    fun = lambda x: VasicekOptimizer(EURIBORFIXMARCH22, result.x[0], result.x[1], x, observedP=straddlesobs.loc[[expiry],[tenor]], observedK=strikes.loc[[expiry],[tenor]])[0],
                    x0  = 1,
                    tol = 1e-15,
                    # bounds = [(1e-17, 1)],
                    options={'gtol': 1e-10, 'disp': True},
                    method = 'L-BFGS-B'
                )
            print(f'\n E: {expiry}, T:{tenor}, vol: {res.x}, error: {res.fun}', end='\n')
            VasiImpVol[tenor].loc[expiry] = res.x[0]
            VasiImpVolPrice[tenor].loc[expiry] = VasicekOptimizer(EURIBORFIXMARCH22, result.x[0], result.x[1], res.x[0], observedP=straddlesobs.loc[[expiry],[tenor]], observedK=strikes.loc[[expiry],[tenor]])[1].values[0][0]

1491.0056917433278                                                            
 E: 1, T:12, vol: [0.00321607], error: 1491.0056866805626


c:\Users\Shark\OneDrive\Desktop\MasterThesis\dynamics.py:155: RuntimeWarning: divide by zero encountered in divide
c:\Users\Shark\OneDrive\Desktop\MasterThesis\dynamics.py:165: RuntimeWarning: divide by zero encountered in divide
c:\Users\Shark\OneDrive\Desktop\MasterThesis\dynamics.py:155: RuntimeWarning: divide by zero encountered in divide
c:\Users\Shark\OneDrive\Desktop\MasterThesis\dynamics.py:165: RuntimeWarning: divide by zero encountered in divide


10966.774573451528                                                            
 E: 1, T:24, vol: [9.91256359e-05], error: 10966.774571762338


c:\Users\Shark\OneDrive\Desktop\MasterThesis\dynamics.py:155: RuntimeWarning: divide by zero encountered in divide
c:\Users\Shark\OneDrive\Desktop\MasterThesis\dynamics.py:165: RuntimeWarning: divide by zero encountered in divide


54539.0148545848                                                               
 E: 1, T:36, vol: [0.00072189], error: 54539.01476962356


...

c:\Users\Shark\OneDrive\Desktop\MasterThesis\dynamics.py:155: RuntimeWarning: divide by zero encountered in divide
c:\Users\Shark\OneDrive\Desktop\MasterThesis\dynamics.py:165: RuntimeWarning: divide by zero encountered in divide


460302.11729279795                                                          
 E: 1, T:180, vol: [0.00027587], error: 460302.1168257531


c:\Users\Shark\OneDrive\Desktop\MasterThesis\dynamics.py:155: RuntimeWarning: divide by zero encountered in divide
c:\Users\Shark\OneDrive\Desktop\MasterThesis\dynamics.py:165: RuntimeWarning: divide by zero encountered in divide


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Shark\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Shark\AppData\Local\Temp\ipykernel_21832\1160685918.py", line 10, in <cell line: 4>
    res = minimize(
  File "c:\Users\Shark\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minimize.py", line 699, in minimize
    res = _minimize_lbfgsb(fun, x0, args, jac, bounds,
  File "c:\Users\Shark\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_lbfgsb_py.py", line 362, in _minimize_lbfgsb
    f, g = func_and_grad(x)
  File "c:\Users\Shark\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_differentiable_functions.py", line 285, in fun_and_grad
    self._update_fun()
  File "c:\Users\Shark\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_differentiable_funct

In [70]:
VasiImpVol

,12,24,36,48,60,84,120,144,180,240,300,360
Expiry,,,,,,,,,,,,
1,3.216063e-03,1.737803e-03,4.508501e-04,1.178609e-03,8.922982e-04,1.542825e-04,3.432925e-04,3.980272e-04,2.758657e-04,3.875859e-04,1.000000e-15,1.000000e-15
3,1.336405e-03,1.888549e-03,1.460823e-04,6.890375e-04,2.221743e-04,8.640043e-05,1.541047e-04,3.207202e-04,2.261957e-04,2.255287e-04,1.000000e-15,1.000000e-15
6,3.309924e-05,2.258404e-03,1.275715e-03,6.326513e-05,5.640727e-04,3.496771e-04,4.330183e-04,3.273040e-04,1.169830e-04,4.621874e-04,1.000000e-15,1.000000e-15
9,2.147388e-02,6.176012e-05,1.295579e-03,9.879811e-04,7.495532e-05,5.730154e-04,5.038093e-04,2.128322e-04,1.899745e-04,1.141971e-04,1.000000e-15,1.000000e-15
12,2.682842e-02,3.447576e-05,9.342423e-04,1.874272e-04,7.878095e-04,8.260789e-05,2.904977e-04,2.615190e-04,2.285638e-04,2.318856e-04,1.000000e-15,1.000000e-15
24,3.830927e-02,2.511295e-02,1.482648e-02,1.050697e-04,1.418540e-04,2.458959e-04,2.876205e-04,7.583189e-05,2.583256e-04,4.685313e-04,1.000000e-15,1.000000e-15
36,4.590906e-02,3.142872e-02,2.395742e-02,1.350782e-02,4.269899e-05,5.653090e-05,3.272823e-04,3.488912e-04,2.193618e-04,4.570674e-04,1.000000e-15,1.000000e-15
48,5.202875e-02,3.627390e-02,2.878677e-02,2.009085e-02,8.824778e-05,7.733831e-04,3.356237e-04,4.611031e-04,3.382575e-04,1.000000e-15,1.000000e-15,1.000000e-15
60,5.766841e-02,4.044704e-02,3.247195e-02,2.361402e-02,6.621679e-04,9.488748e-05,5.011071e-04,3.627836e-04,5.417138e-04,1.000000e-15,1.000000e-15,1.000000e-15


In [69]:
VasiImpVolPrice

,12,24,36,48,60,84,120,144,180,240,300,360
Expiry,,,,,,,,,,,,
1,7.653959e+01,1.489947e+02,2.757167e+02,2.345879e+02,2.711833e+02,3.427410e+02,4.704102e+02,5.625288e+02,7.080415e+02,5.783524e+02,331000000.0,347000000.0
3,8.618616e+01,1.528720e+02,1.989962e+02,2.355530e+02,2.749551e+02,3.508969e+02,4.780659e+02,5.698212e+02,6.625870e+02,5.700620e+02,330000000.0,345800000.0
6,8.441037e+01,1.449382e+02,1.891450e+02,2.269689e+02,2.619184e+02,3.353195e+02,4.695043e+02,5.577236e+02,6.334836e+02,5.319383e+02,329000000.0,344600000.0
9,9.633758e+01,1.348476e+02,1.769722e+02,2.123233e+02,2.510380e+02,3.235960e+02,4.584806e+02,5.432030e+02,6.021631e+02,4.916739e+02,327500000.0,343400000.0
12,1.075229e+02,1.231944e+02,1.602826e+02,1.995439e+02,2.338256e+02,3.100216e+02,4.455960e+02,5.268736e+02,5.813155e+02,4.651695e+02,326500000.0,342200000.0
24,1.358907e+02,1.370303e+02,1.348300e+02,1.595861e+02,1.946773e+02,2.754628e+02,4.167846e+02,4.903858e+02,4.881957e+02,3.352276e+02,322000000.0,337400000.0
36,1.567023e+02,1.572136e+02,1.556935e+02,1.532197e+02,1.868287e+02,2.743549e+02,4.042267e+02,4.746949e+02,4.244606e+02,2.309486e+02,319000000.0,333800000.0
48,1.733324e+02,1.740984e+02,1.712420e+02,1.689281e+02,1.927656e+02,2.871075e+02,3.966488e+02,4.188976e+02,3.601388e+02,3.004000e+08,316000000.0,330200000.0
60,1.874044e+02,1.873152e+02,1.843357e+02,1.812135e+02,2.012505e+02,2.922644e+02,3.957704e+02,3.551689e+02,3.084123e+02,2.980000e+08,312500000.0,327200000.0


In [7]:
import warnings
warnings.filterwarnings('ignore')
#Trying to fit just the first price 1M10Y
from scipy.optimize import minimize, basinhopping
EURIBORFIXMARCH22 = 0.03071
Nfeval = 0
def callback(xk):
    global Nfeval
    print(f'Eval: {Nfeval}, x={xk}, func: {VasicekOptimizer(initial=0.03071, mean=xk[0], reversion=xk[1], volatility=xk[2],observedP=straddlesobs.loc[[12],[120]], observedK=strikes.loc[[12],[120]])[0]}')
    Nfeval += 1
    return True
# callback([0.027832947, 0.11874695, 0.019677])

result = minimize(
    # fun = lambda x: VasicekOptimizer(EURIBORFIXMARCH22, x[0], x[1], x[2], observed=straddlesobs)[0],
    fun = lambda x: VasicekOptimizer(EURIBORFIXMARCH22, x[0], x[1], x[2], observedP=straddlesobs.loc[[12],[120]], observedK=strikes.loc[[12],[120]])[0],
    x0  = [0.06460112, 0.69262944, 0.08495089],
    bounds = [(0.0001,1),(0.0001,1),(0.0001,1)],
    method='nelder-mead',
    callback=callback
)
warnings.filterwarnings('default')

Eval: 0, x=[0.05814101 0.72726091 0.08919843], func: 1292201861.3422172    
Eval: 1, x=[0.05814101 0.72726091 0.08919843], func: 1292201861.3422172    
Eval: 2, x=[0.05383427 0.80806768 0.08636674], func: 949116724.7164184    
Eval: 3, x=[0.04306741 0.8196115  0.09627768], func: 476386538.54004693    
Eval: 4, x=[0.04306741 0.8196115  0.09627768], func: 476386538.54004693    
Eval: 5, x=[0.03517172 0.9119621  0.09297403], func: 153107936.27739525    
Eval: 6, x=[0.03517172 0.9119621  0.09297403], func: 153107936.27739525    
Eval: 7, x=[0.03517172 0.9119621  0.09297403], func: 153107936.27739525    
Eval: 8, x=[0.03517172 0.9119621  0.09297403], func: 153107936.27739525    
Eval: 9, x=[0.0330981  0.88887445 0.09795572], func: 117321160.98053773    
Eval: 10, x=[0.0330981  0.88887445 0.09795572], func: 117321160.98053773   
Eval: 11, x=[0.0298038  0.90234224 0.0985442 ], func: 66301349.27390709    
Eval: 12, x=[0.03106695 0.90651085 0.0980975 ], func: 78743110.92355016    
Eval: 13, x=[

In [8]:
result

 final_simplex: (array([[0.02800237, 1.        , 0.00528899],
       [0.02800232, 1.        , 0.00529918],
       [0.0280024 , 1.        , 0.00529086],
       [0.02800217, 1.        , 0.0052854 ]]), array([2.04108115e-05, 2.74778123e-05, 4.09936893e-05, 9.50589075e-05]))
           fun: 2.0410811520536828e-05
       message: 'Optimization terminated successfully.'
          nfev: 145
           nit: 77
        status: 0
       success: True
             x: array([0.02800237, 1.        , 0.00528899])

In [9]:
VasicekOptimizer(0.03071, 0.02800237, 1, 0.00528899 ,straddlesobs,strikes)[1]

,12,24,36,48,60,84,120,144,180,240,300,360
Expiry,,,,,,,,,,,,
1,58.917636,99.143629,150.949193,110.094709,109.063862,107.415841,130.835772,157.033043,179.918188,52.230429,409.891584,798.131019
3,64.899955,98.146101,107.160772,105.878247,107.728748,110.666857,133.981118,160.056583,160.644366,63.223977,439.342132,830.297717
6,58.753018,84.440820,91.215470,91.267824,88.885507,89.793612,120.677300,143.719512,128.436064,102.750681,466.594634,859.314850
9,52.108070,70.149309,74.665186,72.436010,73.982962,74.511765,106.561936,126.605923,95.696295,142.754626,511.761214,888.887389
12,43.392537,55.825242,55.695596,57.433870,55.117097,59.342307,91.911749,108.961036,74.601642,168.324436,539.853368,918.780054
24,20.054373,25.764327,28.296877,28.493502,28.528450,32.493715,63.708453,74.327112,29.361303,285.947051,669.524366,1038.507869
36,17.301924,23.470767,25.712217,26.463129,26.907245,32.782252,56.708169,65.501229,66.515719,372.878273,746.480569,1117.851970
48,16.827347,22.832385,24.984283,25.903632,27.941888,43.241327,55.829677,30.448854,117.160809,442.625721,819.978913,1193.326417
60,16.366319,22.236839,24.396480,26.336430,32.488497,51.300435,61.428794,38.376173,155.545040,508.703877,904.690252,1246.893843


In [89]:
expiry = 1
tenor  = 12
obsP = straddlesobs[tenor].loc[expiry]
obsK = strikes[tenor].loc[expiry]
def vasiOptVol(initial, mean, rev, vol, expiry, tenor, obsP,obsK):
    print(vol, end='             \r')
    T = np.arange(expiry/12, expiry/12+tenor/12+0.5, 0.5)
    S = np.arange(expiry/12, expiry/12+tenor/12+1)
    Vasi = Vasicek(initial, mean, rev, vol)
    p = Vasi.swaptionSC(0,expiry/12, fixedSchedule=S, floatingSchedule=T, fixedRate=obsK, payer=True)
    r = Vasi.swaptionSC(0,expiry/12, fixedSchedule=S, floatingSchedule=T, fixedRate=obsK, payer=False)
    return abs(obsP-((p+r)*10000))

basinhopping(
    func = lambda v: vasiOptVol(0.03071,0.148,10,v,expiry,tenor,obsP,obsK),
    x0 = 0.5
)

                        fun: 35279.20815855801
 lowest_optimization_result:       fun: 35279.20815855801
 hess_inv: array([[0.0023714]])
      jac: array([31.6875])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 161
      nit: 4
     njev: 75
   status: 2
  success: False
        x: array([0.07514477])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 92
                       nfev: 11133
                        nit: 100
                       njev: 5193
                    success: False
                          x: array([0.07514477])